In [42]:
import pandas as pd
import numpy as np
import os
from os import path
import csv
import tqdm
import networkx as nx

In [43]:
d = os.getcwd()
os.chdir('..')
os.chdir('input_data')

cities        = pd.read_csv('cities.txt', header = None, names = ['cID', 'name', 'X', 'Y', 'w'], index_col = 'cID')
sat_positions = pd.read_csv('sat_positions.txt', header = None, names = ['sID', 'oID', 'soID', 'X', 'Y', 'd'], index_col = 'soID')
city_pairs    = pd.read_csv('city_pairs.txt', header = None, names = ['ID1', 'ID2', 'd'])
city_coverage = pd.read_csv('city_coverage.txt', header = None, names = ['cID', 'sID','l'])
valid_isls    = pd.read_csv('valid_isls.txt', header = None, names = ['sID1', 'sID2', 'l'])

cities = cities.drop(['name', 'X', 'Y'], axis = 1)
sat_positions = sat_positions.drop(['X','Y','d'], axis = 1)

In [44]:
degree = np.zeros(1600)
valid_used = np.zeros(len(valid_isls))

In [45]:
# CREATE GRAPH
G = nx.Graph()
for i in range(1600):
    G.add_node(i)
for i in range(len(valid_isls)):
    G.add_edge(valid_isls['sID1'][i], valid_isls['sID2'][i], weight = valid_isls['l'][i])

In [46]:
i_cov = city_coverage[city_coverage['cID'] == 10002]
print(i_cov)
i_cov.iloc[0]['cID']

      cID   sID           l
10  10002     5  827.245425
11  10002  1055  764.671886
12  10002  1093  660.304296
13  10002  1094  413.094067
14  10002  1133  589.354679
15  10002  1134  543.558810
16  10002  1172  750.968579
17  10002  1173  904.002702
18  10002  1486  750.912187
19  10002  1487  785.374237
20  10002  1526  378.280886
21  10002  1527  695.243138
22  10002  1565  368.846762
23  10002  1566  818.077056


10002.0

In [47]:
for i in range(10001,10101):
    print(i)
    i_cov = city_coverage[city_coverage['cID'] == i]
    for j in range(i+1,10101):
        G_temp = G.copy()
        j_cov = city_coverage[city_coverage['cID'] == j]
        for k in range(len(i_cov)):
            G_temp.add_edge(i_cov.iloc[k]['cID'], i_cov.iloc[k]['sID'], weight = i_cov.iloc[k]['l'])
        for l in range(len(j_cov)):
            G_temp.add_edge(j_cov.iloc[l]['cID'], j_cov.iloc[l]['sID'], weight = j_cov.iloc[l]['l'])
            
        path = nx.dijkstra_path(G_temp, i, j)
        length = len(path)
        #Iterate through whole path
        for m in range(1,length-2):
            if((path[m] < 10000) & (path[m+1] < 10000)):
                u = min(path[m],path[m+1])
                v = max(path[m],path[m+1])
                index = valid_isls[(valid_isls['sID1'] == u) & (valid_isls['sID2'] == v)].index[0]
                valid_used[index] += max(10-length,1)

10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10042
10043
10044
10045
10046
10047
10048
10049
10050
10051
10052
10053
10054
10055
10056
10057
10058
10059
10060
10061
10062
10063
10064
10065
10066
10067
10068
10069
10070
10071
10072
10073
10074
10075
10076
10077
10078
10079
10080
10081
10082
10083
10084
10085
10086
10087
10088
10089
10090
10091
10092
10093
10094
10095
10096
10097
10098
10099
10100


In [48]:
order = np.flip(np.argsort(valid_used))

# CREATE GRAPH
G = nx.Graph()
# Add satellites
for i in range(1600):
    G.add_node(i)
# Add cities
for i in range(100):
    G.add_node(10001 + i)
for i in range(len(order)):
    a = valid_isls['sID1'].iloc[order[i]]
    b = valid_isls['sID2'].iloc[order[i]]
    u = min(a,b)
    v = max(a,b)
    
    if(degree[u] < 4) & (degree[v] < 4):
        G.add_edge(u,v)
        print(u,v)
        degree[u] += 1
        degree[v] += 1

810 1169
49 1290
90 1330
730 1089
690 1050
90 449
770 1130
170 529
256 369
130 490
806 810
49 1571
170 1410
1053 1169
295 731
1169 1486
1093 1169
129 449
374 810
690 806
731 1286
49 1172
1330 1405
296 691
209 529
209 647
169 489
731 810
890 1249
10 1250
490 727
1089 1129
51 253
89 1527
51 645
11 1291
414 529
690 805
295 335
49 1134
854 1286
90 1488
139 645
692 854
1290 1405
1290 1366
295 730
253 255
130 208
854 929
208 605
289 727
208 489
374 771
936 1331
170 605
374 529
449 805
334 371
81 974
255 805
603 644
129 1370
850 1210
91 130
650 1009
690 1010
255 603
256 331
1010 1523
449 727
209 369
169 369
128 1527
1013 1130
414 730
1488 1527
374 730
264 1359
1410 1527
1015 1130
91 607
91 371
128 523
1022 1218
1287 1290
731 844
974 1089
89 1571
1133 1249
737 1359
1330 1485
1291 1571
414 890
322 1022
318 1217
209 1450
135 959
647 805
255 291
334 489
51 130
489 844
170 645
138 566
219 1236
854 968
1134 1173
490 766
1022 1258
770 884
51 208
1130 1327
934 1330
89 1575
140 1235
1089 1168
128 1487

500 535
211 530
177 530
534 924
491 528
492 528
493 528
525 528
420 536
456 536
455 536
454 536
491 533
531 846
531 847
498 533
531 848
531 849
532 891
1598 1599
538 541
233 544
196 544
227 544
228 544
543 1358
543 1481
543 1474
543 1475
115 545
77 545
78 545
110 545
542 1436
542 1398
542 1399
542 1400
113 547
75 547
76 547
77 547
546 1594
546 1595
546 1596
546 1597
72 548
33 548
34 548
35 548
456 538
419 538
420 538
537 1437
537 1438
537 1439
457 539
537 1401
385 539
502 541
464 541
465 541
540 1553
540 1514
540 1515
540 1516
511 1027
511 943
511 944
511 945
353 512
274 512
275 512
276 512
430 510
352 510
509 903
509 906
509 907
509 944
515 1186
279 515
514 1146
514 1147
514 1180
514 1181
240 515
276 515
513 712
513 634
505 1437
505 1355
505 1356
505 1357
30 506
31 506
464 506
465 506
389 504
310 504
503 1552
503 1553
503 1554
503 1555
508 783
275 507
197 507
522 1377
171 527
172 527
173 527
526 728
526 760
526 761
439 518
358 518
359 518
360 518
516 788
516 708
516 709
516 710
365 52

115 1480
116 387
116 388
116 390
114 229
114 231
118 819
118 820
117 193
117 196
117 198
117 199
184 1393
184 1507
184 1509
28 186
183 1354
183 1356
183 1389
183 1390
188 303
188 305
188 306
187 1436
187 1547
187 1549
153 189
29 189
34 189
35 189
178 1036
20 179
177 179
21 180
22 180
23 180
24 180
182 298
182 299
182 300
182 301
181 1121
181 1272
181 1276
181 1312
198 319
200 1138
46 201
47 201
80 201
82 201
155 199
156 199
191 383
191 386
191 387
190 1596
192 305
192 306
190 1595
28 190
153 190
194 820
153 193
154 193
44 161
45 161
46 161
157 898
157 900
157 937
163 279
163 281
162 1256
162 1448
162 1449
162 1451
7 164
154 388
154 390
156 819
156 820
155 233
172 1414
172 1415
166 245
166 281
166 283
165 1376
165 1489
165 1490
165 1491
9 167
34 383
36 819
36 1475
36 1476
36 1477
37 391
25 1031
27 998
27 1032
27 1033
26 1504
26 1585
26 1587
26 1588
28 221
25 1467
25 1469
25 1470
29 70
29 64
46 82
45 1567
45 1569
45 1568
47 50
47 84
44 1175
44 1177
44 1178
39 1363
39 1400
39 1402
2 40
38

874 995
877 995
993 994
994 1115
954 998
997 1034
997 1036
988 1189
988 1188
988 1187
949 988
989 1150
989 1183
989 1184
989 1185
951 987
873 987
870 992
871 992
990 1066
990 1068
990 1069
991 1191
952 991
953 991
954 991
953 957
957 958
960 962
840 961
879 961
961 965
871 951
872 951
877 952
875 952
808 970
969 1044
971 1131
971 1132
972 1088
972 1090
966 1124
966 1126
966 1128
967 1004
829 1145
821 825
824 825
825 826
827 828
828 863
828 865
826 867
826 866
827 1064
827 1066
827 1067
847 848
808 889
821 1018
875 1151
878 1154
878 1191
873 877
888 1164
888 1167
888 1165
856 1051
858 861
858 862
858 1019
848 1165
848 1081
872 1111
873 1026
863 1021
863 1024
863 1058
865 1024
865 1025
1052 1132
1056 1132
1135 1175
1135 1178
1090 1126
1122 1126
1043 1125
1045 1125
1046 1125
1048 1125
1053 1128
1084 1128
1085 1128
1143 1146
1105 1146
1143 1179
1143 1180
1025 1144
1026 1144
1027 1148
1028 1148
1149 1187
1140 1142
1020 1140
1101 1140
1098 1099
1067 1107
1068 1107
1069 1107
1108 1188
1108 11

In [49]:
ISL_PATH = "../output_data/sat_links.txt"
with open(ISL_PATH, mode="w") as isl_file:
    isl_writer = csv.writer(isl_file, delimiter=",")
    for isl in G.edges:
        isl_writer.writerow(isl)